In [240]:
import aocd
import re
from dotenv import load_dotenv
import numpy as np
from tqdm import tqdm
import networkx as nx

load_dotenv()

puzzle = aocd.get_puzzle(day=22, year=2024)
text = puzzle.input_data
examples = puzzle.examples
print(examples)

lines = text.splitlines()

example = examples[0].input_data.splitlines()

examples

[Example(input_data='15887950\n16495136\n527345\n704524\n1553684\n12683156\n11100544\n12249484\n7753432\n5908254', answer_a='37327623', answer_b='9', extra=None)]


[Example(input_data='15887950\n16495136\n527345\n704524\n1553684\n12683156\n11100544\n12249484\n7753432\n5908254', answer_a='37327623', answer_b='9', extra=None)]

In [208]:
text = examples[0].input_data
lines = text.splitlines()

example = """1
2
3
2024"""

# example = """123"""
lines = example.splitlines()

text

'15887950\n16495136\n527345\n704524\n1553684\n12683156\n11100544\n12249484\n7753432\n5908254'

In [241]:
def mix(s1, s2):
    return s1 ^ s2

def prune(s):
    return s % 16777216


def one(s):
    return prune(mix(s, 64*s))

def two(s):
    return prune(mix(s, int(s/32)))

def three(s):
    return prune(mix(s, 2048 * s))

def all(s):
    s = one(s)
    s = two(s)
    s = three(s)
    return s


In [242]:
sum = 0
for s in lines:
    s = int(s)
    for _ in range(2000):
        s = all(s)
    sum += s

print(sum)

14392541715


In [39]:
aocd.submit(sum, day=22, year=2024)

answer a: None
submitting for part a


That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


Gtk-Message: 10:31:35.144: Not loading module "atk-bridge": The functionality is provided by GTK natively. Please try to not load it.


In [243]:
digits = np.zeros((len(lines), 2001), dtype=int)

for i, s in enumerate(lines):
    digits[i, 0] = int(s[-1])
    s = int(s)
    
    for j in range(1,2001):
        s = all(s)
        digits[i, j] = int(str(s)[-1])

print(digits)

[[2 4 0 ... 8 5 0]
 [4 0 9 ... 1 1 7]
 [3 1 7 ... 2 4 5]
 ...
 [4 9 5 ... 0 7 0]
 [4 4 3 ... 4 1 7]
 [9 4 5 ... 3 4 8]]


In [244]:
diff = np.diff(digits, axis=1)
diff.shape

(1691, 2000)

In [245]:
seqs = set()

for d in diff:
    for i in range(1997):
        t = tuple(d[i:i+4].tolist())
        assert len(t) == 4
        seqs.add(t)




seqs = list(seqs)

In [214]:
diff.shape

(4, 2000)

In [215]:
"-2,1,-1,3" in ",".join(map(str,diff[1]))

True

In [246]:
import torch

unrolled = torch.tensor(diff).unfold(1, 4, 1)
print(unrolled.shape)

m, s = unrolled, torch.tensor(digits)[:, 4:] 
print(m.shape, s.shape)

s0_t = torch.tensor((-2,1,-1,3))
print(s0_t)

r = torch.arange(1997, 0, -1)[None]

mask = (m == s0_t).all(dim=2)
print(mask.shape)
print(mask.sum(), mask.any(1))

amax = (mask * r).argmax(1)
print(amax)
amax_mask = amax > 0    
s[amax_mask, amax[amax_mask]]




torch.Size([1691, 1997, 4])
torch.Size([1691, 1997, 4]) torch.Size([1691, 1997])
tensor([-2,  1, -1,  3])
torch.Size([1691, 1997])
tensor(225) tensor([False, False, False,  ..., False,  True, False])
tensor([   0,    0,    0,  ...,    0, 1717,    0])


tensor([4, 4, 4, 5, 8, 4, 6, 3, 4, 8, 7, 7, 3, 3, 9, 6, 4, 5, 7, 5, 5, 7, 7, 3,
        8, 5, 5, 8, 5, 8, 3, 4, 6, 3, 5, 5, 7, 8, 4, 9, 4, 4, 5, 6, 9, 4, 5, 6,
        9, 9, 3, 9, 8, 6, 9, 7, 5, 6, 3, 9, 6, 7, 9, 8, 9, 5, 9, 3, 8, 3, 4, 5,
        4, 7, 4, 8, 6, 3, 5, 5, 6, 6, 6, 6, 5, 4, 6, 7, 8, 7, 6, 5, 6, 6, 9, 4,
        5, 9, 3, 4, 8, 5, 5, 7, 5, 3, 8, 7, 4, 5, 7, 8, 9, 4, 8, 7, 7, 6, 8, 5,
        7, 6, 3, 9, 9, 6, 6, 7, 8, 4, 8, 9, 8, 8, 3, 5, 3, 3, 6, 8, 5, 5, 4, 8,
        4, 3, 3, 5, 5, 4, 8, 7, 5, 6, 3, 6, 9, 6, 8, 6, 6, 7, 5, 9, 4, 4, 5, 3,
        7, 5, 7, 5, 4, 8, 4, 5, 6, 4, 5, 9, 5, 8, 7, 6, 8, 8, 3, 5, 8, 7, 8, 3,
        3, 5, 8, 4, 7, 8, 6, 9, 8, 8, 5, 3, 8, 4, 3, 6, 6, 5, 6, 8])

In [224]:
diff[1, 287:287+5], m[1, 287]

(array([-2,  1, -1,  3,  1]), tensor([-2,  1, -1,  3]))

In [247]:
from tqdm import tqdm

s0 = torch.tensor(seqs[0])

sums, selected_set = [], []
for s0 in tqdm(seqs):

    s0_t = torch.tensor(s0)
    mask = (m == s0_t).all(dim=2)
    amax = (mask * r).argmax(1)
    amax_mask = amax > 0
    ss = s[amax_mask, amax[amax_mask]].sum()
    sums.append(ss)
    if s0 == (-2,1,-1,3):
        print(ss)

    selected_set.append(s0)

print(max(zip(sums, selected_set)))

 37%|███▋      | 15102/40951 [03:12<06:28, 66.59it/s]

tensor(1253)


100%|██████████| 40951/40951 [11:04<00:00, 61.63it/s]


(tensor(1628), (-1, 2, -2, 4))


In [248]:
aocd.submit(max(sums).item(), day=22, year=2024)

answer a: 14392541715
submitting for part b (part a is already completed)


That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 22! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


Gtk-Message: 14:58:03.332: Not loading module "atk-bridge": The functionality is provided by GTK natively. Please try to not load it.
